In [ ]:
cd D:/Documents/CODING/AI/A.I_tailor

In [ ]:
#!git clone https://github.com/facebookresearch/pifuhd

In [ ]:
#!git clone https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git

In [ ]:
#!pip install rembg

In [ ]:
cd D:/Documents/CODING/AI/A.I_tailor/pifuhd/sample_images

In [ ]:
filename0 = 'test.png'

In [ ]:
import rembg
file_name = filename0
with open(file_name, "rb") as input_file, open(f"output_{file_name}", "wb") as output_file:
    img = rembg.remove(input_file.read())
    output_file.write(img)

In [ ]:
output_test


In [ ]:
filename = 'output_%s' % filename0

In [ ]:
import os
try:
  image_path = 'D:/Documents/CODING/AI/A.I_tailor/pifuhd/sample_images/%s' % filename
except:
  image_path = 'D:/Documents/CODING/AI/A.I_tailor/pifuhd/sample_images/test.png' # example image
image_dir = os.path.dirname(image_path)
file_name = os.path.splitext(os.path.basename(image_path))[0]
# output pathes
obj_path = 'D:/Documents/CODING/AI/A.I_tailor/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name
out_img_path = 'D:/Documents/CODING/AI/A.I_tailor/pifuhd/results/pifuhd_final/recon/result_%s_256.png' % file_name
video_path = 'D:/Documents/CODING/AI/A.I_tailor/pifuhd/results/pifuhd_final/recon/result_%s_256.mp4' % file_name
video_display_path = 'D:/Documents/CODING/AI/A.I_tailor/pifuhd/results/pifuhd_final/result_%s_256_display.mp4' % file_name

In [ ]:
cd D:/Documents/CODING/AI/A.I_tailor

In [ ]:
cd D:/Documents/CODING/AI/A.I_tailor/lightweight-human-pose-estimation.pytorch/

In [ ]:
# import requests
# url = "https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth"
# output_file = "checkpoint_iter_370000.pth"
# response = requests.get(url)
# if response.status_code == 200:
#     with open(output_file, "wb") as file:
#         file.write(response.content)
#     print(f"Downloaded {output_file}")
# else:
#     print("Failed to download the file.")

In [ ]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo
def get_rect(net, images, height_size):
    net = net.eval()
    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)
        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)
        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []
        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)
            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)
              center = (0.5 * (pmax[:2] + pmin[:2])).astype(int)
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)
            x1 = center[0] - radius
            y1 = center[1] - radius
            rects.append([x1, y1, 2*radius, 2*radius])
        np.savetxt(rect_path, np.array(rects), fmt='%d')

In [ ]:
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)
get_rect(net.cuda(), [image_path], 512)

In [ ]:
cd D:/Documents/CODING/AI/A.I_tailor/pifuhd/

In [ ]:
#import subprocess
# Command to execute the shell script using Command Prompt
#script_path = "D:/Documents/CODING/AI/A.I_tailor/pifuhd/scripts/download_trained_model.sh"
#subprocess.run(script_path, shell=True)

## Run PIFuHD!


In [ ]:
# Warning: all images with the corresponding rectangle files under -i will be processed.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

!python -m apps.simple_test -r 256 --use_rect -i $image_dir
# seems that 256 is the maximum resolution that can fit into Google Colab.
# If you want to reconstruct a higher-resolution mesh, please try with your own machine.

In [ ]:
cd D:/Documents/CODING/AI/A.I_tailor/pifuhd/results/pifuhd_final/recon

In [ ]:
import trimesh
import numpy as np
# Load the 3D mesh in OBJ format
mesh = trimesh.load('D:/Documents/CODING/AI/A.I_tailor/pifuhd/results/pifuhd_final/recon/result_%s_256.obj' % file_name)
vertices = mesh.vertices
n_points = 2048  # You can adjust the number of points as needed
sampled_indices = np.random.choice(len(vertices), n_points, replace=False)
point_cloud = vertices[sampled_indices]
x = point_cloud[:, 0]
y = point_cloud[:, 1]
z = point_cloud[:, 2]
point_cloud_ply = trimesh.Trimesh(vertices=point_cloud)
point_cloud_ply.export('point_cloud.ply')
print("Point cloud generated and saved as 'point_cloud.ply'")
